# Explores Weather Information in the US

Clean Data

In [1]:
with open("artifacts/ghcnd-stations.txt") as bad_stations, open("artifacts/ghcnd-stations-fixed.txt", 'w+') as good_stations:
    for b in bad_stations:
        good_stations.write("[" + '[,['.join(b[:74].split(None, 4)) + "[")
        good_stations.write("\n")


Read Data

In [2]:
import pandas as pd
import numpy as np

# https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/GHCND_documentation.pdf
with open("artifacts/ghcnd-stations-fixed.txt") as reader:
    ghcnd_stations = pd.read_csv(
        reader,
        header=None,
        quotechar='[',
        sep=',',
        names=["station_id", "latitude", "longitude", "elevation_above_sea_level", "station_name"]
    )

ghcnd_stations

,station_id,latitude,longitude,elevation_above_sea_level,station_name
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP GS
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL
...,...,...,...,...,...
118487,ZI000067969,-21.0500,29.3670,861.0,WEST NICHOLSON
118488,ZI000067975,-20.0670,30.8670,1095.0,MASVINGO
118489,ZI000067977,-21.0170,31.5830,430.0,BUFFALO RANGE
118490,ZI000067983,-20.2000,32.6160,1132.0,CHIPINGE GS


Filter only US weather stations.

In [3]:
ghcnd_stations.query("station_id == 'US1WAKG0196'")

,station_id,latitude,longitude,elevation_above_sea_level,station_name
89111,US1WAKG0196,47.3539,-122.3181,94.8,WA DES MOINES 2.9 S


In [4]:
us_stations = ghcnd_stations[ghcnd_stations['station_id'].str.startswith('US')]
us_stations

,station_id,latitude,longitude,elevation_above_sea_level,station_name
52532,US009052008,43.7333,-96.6333,482.0,SD SIOUX FALLS (ENVIRON. CANADA)
52533,US10RMHS145,40.5268,-105.1113,1569.1,CO RMHS 1.6 SSW
52534,US10adam001,40.5680,-98.5069,598.0,NE JUNIATA 1.5 S
52535,US10adam002,40.5093,-98.5493,601.1,NE JUNIATA 6.0 SSW
52536,US10adam003,40.4663,-98.6537,615.1,NE HOLSTEIN 0.1 NW
...,...,...,...,...,...
117697,USW00096405,60.4731,-145.3542,25.3,AK CORDOVA 14 ESE
117698,USW00096406,64.5014,-154.1297,78.9,AK RUBY 44 ESE
117699,USW00096407,66.5620,-159.0036,6.7,AK SELAWIK 28 E
117700,USW00096408,63.4519,-150.8747,678.2,AK DENALI 27 N


Reusable functions.

In [5]:
def coordinate_msg(latitude: float, longitude: float):
    if latitude > 0:
        latitude_msg = f"{latitude}°N"
    elif latitude < 0:
        latitude_msg = f"{abs(latitude)}°S"
    else:
        latitude_msg = "0°"

    if longitude > 0:
        longitude_msg = f"{longitude}°E"
    elif longitude < 0:
        longitude_msg = f"{abs(longitude)}°W"
    else:
        longitude_msg = "0°"


    return ','.join([latitude_msg, longitude_msg])

For each of the weather stations, pick the closest zip code.

In [6]:
from uszipcode import SearchEngine


def find_closest_zip_code(latitude: float, longitude: float):
    search = SearchEngine()
    result = search.by_coordinates(lat=latitude, lng=longitude, radius=5)

    if len(result) == 0:
        return pd.NA
    else:
        return result[0].zipcode
    

us_stations['zip_code'] =  np.vectorize(find_closest_zip_code)(us_stations['latitude'], us_stations['longitude'])
us_stations

/var/folders/44/y4yg1dfd20sg9qvqvp_3djc40000gn/T/ipykernel_32653/1598789507.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_stations['zip_code'] =  np.vectorize(find_closest_zip_code)(us_stations['latitude'], us_stations['longitude'])


,station_id,latitude,longitude,elevation_above_sea_level,station_name,zip_code
52532,US009052008,43.7333,-96.6333,482.0,SD SIOUX FALLS (ENVIRON. CANADA),<NA>
52533,US10RMHS145,40.5268,-105.1113,1569.1,CO RMHS 1.6 SSW,80526
52534,US10adam001,40.5680,-98.5069,598.0,NE JUNIATA 1.5 S,68955
52535,US10adam002,40.5093,-98.5493,601.1,NE JUNIATA 6.0 SSW,68973
52536,US10adam003,40.4663,-98.6537,615.1,NE HOLSTEIN 0.1 NW,68950
...,...,...,...,...,...,...
117697,USW00096405,60.4731,-145.3542,25.3,AK CORDOVA 14 ESE,<NA>
117698,USW00096406,64.5014,-154.1297,78.9,AK RUBY 44 ESE,<NA>
117699,USW00096407,66.5620,-159.0036,6.7,AK SELAWIK 28 E,<NA>
117700,USW00096408,63.4519,-150.8747,678.2,AK DENALI 27 N,<NA>


Filter where Zip Code is NA

In [7]:
us_stations_all_zip = us_stations[us_stations['zip_code'].notna()]
us_stations_all_zip

,station_id,latitude,longitude,elevation_above_sea_level,station_name,zip_code
52533,US10RMHS145,40.5268,-105.1113,1569.1,CO RMHS 1.6 SSW,80526
52534,US10adam001,40.5680,-98.5069,598.0,NE JUNIATA 1.5 S,68955
52535,US10adam002,40.5093,-98.5493,601.1,NE JUNIATA 6.0 SSW,68973
52536,US10adam003,40.4663,-98.6537,615.1,NE HOLSTEIN 0.1 NW,68950
52537,US10adam004,40.4798,-98.4026,570.0,NE AYR 3.5 NE,68925
...,...,...,...,...,...,...
117691,USW00094992,47.7472,-90.3444,185.9,MN GRAND MARAIS,55604
117692,USW00094993,45.6689,-96.9914,353.9,SD SISSETON MUNI AP,57262
117693,USW00094994,43.1561,-90.6775,204.8,WI BOSCOBEL AP,53805
117694,USW00094995,40.8483,-96.5650,362.4,NE LINCOLN 8 ENE,68527


In [8]:
us_stations_all_zip.station_id.to_list()

['US10RMHS145',
 'US10adam001',
 'US10adam002',
 'US10adam003',
 'US10adam004',
 'US10adam006',
 'US10adam007',
 'US10adam008',
 'US10adam010',
 'US10adam011',
 'US10adam012',
 'US10adam015',
 'US10adam016',
 'US10adam019',
 'US10adam022',
 'US10adam023',
 'US10adam025',
 'US10adam027',
 'US10adam028',
 'US10adam029',
 'US10adam032',
 'US10adam034',
 'US10adam035',
 'US10adam036',
 'US10adam039',
 'US10adam041',
 'US10adam043',
 'US10adam045',
 'US10adam046',
 'US10adam047',
 'US10adam051',
 'US10adam053',
 'US10adam054',
 'US10adam056',
 'US10ante003',
 'US10bann006',
 'US10bann007',
 'US10blai004',
 'US10blai012',
 'US10boon004',
 'US10boon005',
 'US10boon006',
 'US10boon008',
 'US10boon011',
 'US10box_005',
 'US10box_006',
 'US10box_008',
 'US10box_013',
 'US10box_020',
 'US10box_024',
 'US10boyd002',
 'US10boyd003',
 'US10boyd005',
 'US10brow001',
 'US10brow003',
 'US10brow004',
 'US10brow006',
 'US10buff001',
 'US10buff002',
 'US10buff004',
 'US10buff006',
 'US10buff007',
 'US10bu

For each weather station, call the API to get the correct csv files.

E.g. `https://www.ncei.noaa.gov/data/global-summary-of-the-month/access/ACW00011647.csv`

**Warning:** Takes avg 74 mins to get all files.

In [9]:
from concurrent.futures import ThreadPoolExecutor
import requests


def generate_dl_kwargs(limit: int = None):
    return [
        us_station
        for us_station in us_stations_all_zip.station_id.to_list()
    ][:limit]


def download_file(us_station: str):
    filename = f"{us_station}.csv"
    url = f"https://www.ncei.noaa.gov/data/global-summary-of-the-month/access/{filename}"
    print(f"Downloading csv file from {url}")
    response = requests.get(url)
    if str(response.status_code).startswith(('4', '5')):
        print(f"Failed to download file {filename}")
    else:
        with open(f"artifacts/access/{filename}", mode="wb") as file:
            file.write(response.content)
        print(f"Downloaded file {filename}")

# Commented out, don't download again
# with ThreadPoolExecutor(max_workers=10) as executor:
#     executor.map(download_file, generate_dl_kwargs())

Filter 2015 - 2023. Stitch all csv files together.

In [10]:
# Commented out, don't write again
import pathlib

all_csvs = pathlib.Path('artifacts/access').glob("*.csv")
us_data = pd.concat([pd.read_csv(c) for c in all_csvs], ignore_index=True)

In [11]:
us_data['DATE'] = pd.to_datetime(us_data['DATE'], errors="ignore")
us_data_filtered_df = us_data[us_data['DATE'].dt.year >= 2015]

/var/folders/44/y4yg1dfd20sg9qvqvp_3djc40000gn/T/ipykernel_32653/2213275545.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  us_data['DATE'] = pd.to_datetime(us_data['DATE'], errors="ignore")


In [12]:
us_data_filtered_df

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,DP01,DP01_ATTRIBUTES,DP10,DP10_ATTRIBUTES,...,MX05,MX05_ATTRIBUTES,MX06,MX06_ATTRIBUTES,MX07,MX07_ATTRIBUTES,MX08,MX08_ATTRIBUTES,MX09,MX09_ATTRIBUTES
0,US1MDFR0038,2019-08-01,39.419264,-77.430926,100.0,"FREDERICK 1.0 SW, MD US",9.0,",N",5.0,",N",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1MDFR0038,2019-09-01,39.419264,-77.430926,100.0,"FREDERICK 1.0 SW, MD US",5.0,",N",3.0,",N",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US1MDFR0038,2019-10-01,39.419264,-77.430926,100.0,"FREDERICK 1.0 SW, MD US",12.0,",N",10.0,",N",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US1MDFR0038,2019-12-01,39.419264,-77.430926,100.0,"FREDERICK 1.0 SW, MD US",14.0,"1,N",8.0,"1,N",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US1MDFR0038,2020-01-01,39.419264,-77.430926,100.0,"FREDERICK 1.0 SW, MD US",11.0,",N",6.0,",N",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9420654,US1MIHG0017,2017-12-01,47.253000,-88.446521,368.2,"CALUMET 0.5 NE CALUMET LAKE, MI US",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9420655,US1MIHG0017,2018-01-01,47.253000,-88.446521,368.2,"CALUMET 0.5 NE CALUMET LAKE, MI US",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9420656,US1MIHG0017,2019-01-01,47.253000,-88.446521,368.2,"CALUMET 0.5 NE CALUMET LAKE, MI US",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9420657,US1MIHG0017,2019-02-01,47.253000,-88.446521,368.2,"CALUMET 0.5 NE CALUMET LAKE, MI US",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
us_data_filtered_df.to_csv(
    "artifacts/us_data.csv",
    index=False
)

In [ ]:
us_stations_all_zip.to_csv(
    "artifacts/us_data_all_zip.csv",
    index=False
)